In [10]:
molecule1 = 'Li 0.0 0.0 0.0; H 0.0 0.0 1.5474'
molecule2 = 'Li 0.0 0.0 0.0; H 0.0 0.0 1.54'
m=[molecule1, molecule2]

In [16]:
for i in range(len(m)):
    molecule = m[i]

    from qiskit_nature.drivers import PySCFDriver
    from qiskit_nature.transformers import FreezeCoreTransformer

    driver = PySCFDriver(atom=molecule)
    qmolecule = driver.run()

    from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
    problem = ElectronicStructureProblem(driver)

    from qiskit_nature.mappers.second_quantization import ParityMapper
    from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter
#from qiskit.aqua.operators import Z2Symmetries

    converter = QubitConverter(mapper=ParityMapper(), two_qubit_reduction=True)

    from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
    from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
    import numpy as np 

    def exact_diagonalizer(problem, converter):
        solver = NumPyMinimumEigensolverFactory()
        calc = GroundStateEigensolver(converter, solver)
        result = calc.solve(problem)
        return result

    result_exact = exact_diagonalizer(problem, converter)
    exact_energy = np.real(result_exact.eigenenergies[0])

    from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
    fct=[FreezeCoreTransformer(freeze_core=True,remove_orbitals=[2,4])]
    problem = ElectronicStructureProblem(driver, q_molecule_transformers=fct)
# Generate the second-quantized operators
    second_q_ops = problem.second_q_ops()

# Hamiltonian
    main_op = second_q_ops[0]

    from qiskit_nature.mappers.second_quantization import ParityMapper
    from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter
#from qiskit.aqua.operators import Z2Symmetries

    converter = QubitConverter(mapper=ParityMapper(), two_qubit_reduction=True)

# The fermionic operators are mapped to qubit operators
    num_particles = (problem.molecule_data_transformed.num_alpha,
                 problem.molecule_data_transformed.num_beta)
    qubit_op = converter.convert(main_op, num_particles=num_particles)

    from qiskit_nature.circuit.library import HartreeFock

    num_particles = (problem.molecule_data_transformed.num_alpha,
                 problem.molecule_data_transformed.num_beta)
    num_spin_orbitals = 2 * problem.molecule_data_transformed.num_molecular_orbitals
    init_state = HartreeFock(num_spin_orbitals, num_particles, converter)

    from qiskit.circuit.library import TwoLocal

# Choose the ansatz
    ansatz_type = "TwoLocal"

# Parameters for q-UCC antatze
    num_particles = (problem.molecule_data_transformed.num_alpha,problem.molecule_data_transformed.num_beta)
    num_spin_orbitals = 2 * problem.molecule_data_transformed.num_molecular_orbitals

# Put arguments for twolocal
    if ansatz_type == "TwoLocal":
    # Single qubit rotations that are placed on all qubits with independent parameters
        rotation_blocks = ['ry', 'rz']
    # Entangling gates
        entanglement_blocks = 'cx'
    # How the qubits are entangled 
        entanglement = 'linear'
    # Repetitions of rotation_blocks + entanglement_blocks with independent parameters
        repetitions = 1
    # Skip the final rotation_blocks layer
        skip_final_rotation_layer = True
        ansatz = TwoLocal(qubit_op.num_qubits, rotation_blocks, entanglement_blocks, reps=repetitions,entanglement=entanglement, skip_final_rotation_layer=skip_final_rotation_layer)
    # Add the initial state
        ansatz.compose(init_state, front=True, inplace=True)

    from qiskit import Aer
    backend = Aer.get_backend('statevector_simulator')

    from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP

    optimizer_type = 'L_BFGS_B'

# You may want to tune the parameters 
# of each optimizer, here the defaults are used
    if optimizer_type == 'COBYLA':
        optimizer = COBYLA(maxiter=500)
    elif optimizer_type == 'L_BFGS_B':
        optimizer = L_BFGS_B(maxfun=1500)
    elif optimizer_type == 'SPSA':
        optimizer = SPSA(maxiter=500)
    elif optimizer_type == 'SLSQP':
        optimizer = SLSQP(maxiter=500)


#print(result_exact)
# Check with your VQE result.

    from qiskit.algorithms import VQE
    from IPython.display import display, clear_output

# Print and save the data in lists
    def callback(eval_count, parameters, mean, std):  
    # Overwrites the same line when printing
        display("Evaluation: {}, Energy: {}, Std: {}".format(eval_count, mean, std))
        clear_output(wait=True)
        counts.append(eval_count)
        values.append(mean)
        params.append(parameters)
        deviation.append(std)

    counts = []
    values = []
    params = []
    deviation = []

# Set initial parameters of the ansatz
# We choose a fixed small displacement 
# So all participants start from similar starting point
    try:
        initial_point = [0.01] * len(ansatz.ordered_parameters)
    except:
        initial_point = [0.01] * ansatz.num_parameters

    algorithm = VQE(ansatz,
                optimizer=optimizer,
                quantum_instance=backend,
                callback=callback,
                initial_point=initial_point)

    #result = algorithm.compute_minimum_eigenvalue(qubit_op)

# Store results in a dictionary
    from qiskit.transpiler import PassManager
    from qiskit.transpiler.passes import Unroller

# Unroller transpile your circuit into CNOTs and U gates
    pass_ = Unroller(['u', 'cx'])
    pm = PassManager(pass_)
    ansatz_tp = pm.run(ansatz)
    cnots = ansatz_tp.count_ops()['cx']
    score = cnots

    accuracy_threshold = 4.0 # in mHa
    energy = result.optimal_value

    print("For", m[i], ":")
    
# Check your answer using following code
    from qc_grader import grade_ex5
    freeze_core = True # change to True if you freezed core electrons
    grade_ex5(ansatz,qubit_op,result,freeze_core)
    print("-----------------------------------------------------")

For Li 0.0 0.0 0.0; H 0.0 0.0 1.5474 :
Grading your answer for ex5. Please wait...

Oops 😕! Calculated energy (-8.90305) is higher than the threshold (-8.90447)
Please review your answer and try again.
3
-1.0860250074576832
-----------------------------------------------------
For Li 0.0 0.0 0.0; H 0.0 0.0 1.54 :
Grading your answer for ex5. Please wait...

Congratulations 🎉! Your answer is correct.
Your cost is 3.
Feel free to submit your answer.

3
-1.0860250074576832
-----------------------------------------------------


In [107]:
# Submit your answer. You can re-submit at any time.
from qc_grader import submit_ex5
submit_ex5(ansatz,qubit_op,result,freeze_core)

Submitting your answer for ex5. Please wait...
Success 🎉! Your answer has been submitted.


## Additional information

**Created by:** Igor Sokolov, Junye Huang, Rahul Pratap Singh

**Version:** 1.0.0